In [2]:
import pandas as pd 
import numpy as np 
#!pip install OSMnx
import osmnx as ox 
import geopandas as gpd 

In [3]:
df = pd.read_csv('./train.csv')
df.head()


C:\Users\rudra\AppData\Local\Temp\ipykernel_7484\3933076151.py:1: DtypeWarning: Columns (11,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./train.csv')


,id,latitude,longitude,thumb_original_url,country,sequence,captured_at,lon_bin,lat_bin,cell,...,quadtree_10_50000,quadtree_10_12500,quadtree_10_500,quadtree_10_2500,unique_region,unique_sub-region,unique_city,unique_country,creator_username,creator_id
0,3859149887465501,-43.804769,-176.614093,https://scontent-cdg4-1.xx.fbcdn.net/m1/v/t6/A...,NZ,1qOCfogkQ5uE4ZvEf_4n2A,1547559856000,0,8,"(0, 8)",...,0,0,0,0,Chatham Islands_NZ,NaN,Waitangi_NaN_Chatham Islands_NZ,NZ,roadroid,1.113362e+14
1,574181207305439,-43.796611,-176.660483,https://scontent-cdg4-1.xx.fbcdn.net/m1/v/t6/A...,NZ,5rdtj6tui6qdlg8q812ul2,1567257056000,0,8,"(0, 8)",...,0,0,0,0,Chatham Islands_NZ,NaN,Waitangi_NaN_Chatham Islands_NZ,NZ,roadroid,1.113362e+14
2,333574322129026,-43.818092,-176.578383,https://scontent-cdg4-1.xx.fbcdn.net/m1/v/t6/A...,NZ,nJcst1M2bFUxSOA54CZiy9,1531231715132,0,8,"(0, 8)",...,0,0,0,0,Chatham Islands_NZ,NaN,Waitangi_NaN_Chatham Islands_NZ,NZ,roadroid,1.113362e+14
3,636305258168031,-44.052910,-176.633065,https://scontent-cdg4-1.xx.fbcdn.net/m1/v/t6/A...,NZ,BGSwcf0pLCE5bMUWdKTeqk,1662645171414,0,8,"(0, 8)",...,0,0,0,0,Chatham Islands_NZ,NaN,Waitangi_NaN_Chatham Islands_NZ,NZ,roadroid,1.113362e+14
4,166741299029322,-43.748077,-176.329626,https://scontent-cdg4-1.xx.fbcdn.net/m1/v/t6/A...,NZ,05pt8HKnLCf47UTGPrxuzB,1531143464444,0,8,"(0, 8)",...,0,0,0,0,Chatham Islands_NZ,NaN,Waitangi_NaN_Chatham Islands_NZ,NZ,roadroid,1.113362e+14


In [10]:
california_subset = df[df['region'] == 'California'][['id', 'latitude', 'longitude', 'region']].reset_index(drop=True)
california_subset.head(10)


,id,latitude,longitude,region
0,482881322835001,32.576605,-117.103681,California
1,908534226605339,32.612058,-117.134421,California
2,209827997640609,32.580245,-117.130901,California
3,770226430314780,32.580246,-117.118403,California
4,300504424995118,32.582870,-117.104421,California
5,1711449159056882,32.579125,-117.105702,California
6,312230600303209,32.578991,-117.116704,California
7,965984940824691,32.576619,-117.128860,California
8,294804472216861,32.535331,-117.103633,California
9,1106209253193273,32.525260,-117.116146,California


In [11]:
california_subset.info

<bound method DataFrame.info of                       id   latitude   longitude      region
0        482881322835001  32.576605 -117.103681  California
1        908534226605339  32.612058 -117.134421  California
2        209827997640609  32.580245 -117.130901  California
3        770226430314780  32.580246 -117.118403  California
4        300504424995118  32.582870 -117.104421  California
...                  ...        ...         ...         ...
140648   661284462243824  41.762162 -120.375906  California
140649  1516521418828253  41.754005 -120.375215  California
140650   467095782189668  41.702665 -120.375207  California
140651  4399781830146441  41.752781 -120.375509  California
140652  1537945353320808  41.667479 -120.386771  California

[140653 rows x 4 columns]>

In [12]:
id_10k = california_subset.loc[:9999, 'id']
latitude_10k = california_subset.loc[:9999, 'latitude']
longitude_10k = california_subset.loc[:9999, 'longitude']

In [13]:
print("Lat:", latitude_10k[:10])
print("Lon:", longitude_10k[:10])

Lat: 0    32.576605
1    32.612058
2    32.580245
3    32.580246
4    32.582870
5    32.579125
6    32.578991
7    32.576619
8    32.535331
9    32.525260
Name: latitude, dtype: float64
Lon: 0   -117.103681
1   -117.134421
2   -117.130901
3   -117.118403
4   -117.104421
5   -117.105702
6   -117.116704
7   -117.128860
8   -117.103633
9   -117.116146
Name: longitude, dtype: float64


In [15]:
radius = 30
tags = {'amenity': ['post_box', 'bicycle_parking', 'bench', 'fast_food', 'waste_basket']}
results = []
for id, lat, lon in zip(id_10k, latitude_10k, longitude_10k):
    try:
        # Fetch amenities near the point
        location_point = (lat, lon)
        gdf = ox.features_from_point(location_point, tags, dist=radius)
        
        # Extract relevant information (amenities and point metadata)
        for idx, row in gdf.iterrows():
            results.append({
                'ID': id,
                'Latitude': lat,
                'Longitude': lon,
                'Amenity': row.get('amenity', 'Unknown'), 
                'Name': row.get('name', 'Unknown')        
            })
    except Exception as e:
        print(f"Error fetching amenities for ID {id}: {e}")

# Convert results to a DataFrame
amenities_df = pd.DataFrame(results)

# Display the DataFrame
print(amenities_df)


amenities_df.to_csv("amenities_near_points.csv", index=False)
    

Error fetching amenities for ID 482881322835001: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 908534226605339: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 209827997640609: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 770226430314780: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 300504424995118: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 1711449159056882: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 312230600303209: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 965984940824691: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 294804472216861: No matching features. Check query location, tags, and log.
Error fetching amenities fo

c:\Users\rudra\AppData\Local\Programs\Python\Python311\Lib\site-packages\osmnx\features.py:690: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  gdf.loc[:, "geometry"] = gdf["geometry"].make_valid()


Error fetching amenities for ID 281092993709700: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 281453617007118: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 802756617283549: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 506556330546307: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 1636263243428645: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 3967045410011821: No matching features. Check query location, tags, and log.


c:\Users\rudra\AppData\Local\Programs\Python\Python311\Lib\site-packages\osmnx\features.py:690: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  gdf.loc[:, "geometry"] = gdf["geometry"].make_valid()


Error fetching amenities for ID 321854055972871: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 523573815668408: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 974716160022002: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 315178783340941: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 842100443072359: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 1556391704566676: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 169751168359351: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 277980513985419: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 242710287634357: No matching features. Check query location, tags, and log.
Error fetching amenities fo

...

Error fetching amenities for ID 792393608055494: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 445910240088239: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 477529606912878: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 398873122342458: No matching features. Check query location, tags, and log.


c:\Users\rudra\AppData\Local\Programs\Python\Python311\Lib\site-packages\osmnx\features.py:690: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  gdf.loc[:, "geometry"] = gdf["geometry"].make_valid()


Error fetching amenities for ID 1573450716181548: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 307970430877028: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 768735220507433: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 2788778968099601: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 319343883676783: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 402003681587266: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 524846785185038: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 1193619707739738: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 458178401939440: No matching features. Check query location, tags, and log.
Error fetching amenities 

c:\Users\rudra\AppData\Local\Programs\Python\Python311\Lib\site-packages\osmnx\features.py:690: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  gdf.loc[:, "geometry"] = gdf["geometry"].make_valid()


Error fetching amenities for ID 1125682621242357: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 557959558784004: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 4763252813698810: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 188289366478828: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 173779691292516: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 508430653672147: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 537763997309265: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 1087725781749963: No matching features. Check query location, tags, and log.
Error fetching amenities for ID 519227679118224: No matching features. Check query location, tags, and log.
Error fetching amenities 